In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import random
import mofr
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier, plot_tree
from lightgbm import LGBMClassifier, plot_tree #LGBMRanker
import math
import pickle
import statsmodels.api as sm


import xgboost as xgb

In [3]:
seed=1234

# Loading data

In [4]:
import os
os.getcwd()

'C:\\Users\\AZUBA\\Documents\\Data_science\\Kaggle\\8.March ML Mania 2023'

In [5]:
MMasseyOrdinals=pd.read_csv(os.getcwd()+r'\relevant_data_updated\MMasseyOrdinals.csv')

MNCAATourneyCompactResults=pd.read_csv(os.getcwd()+r'\relevant_data_updated\MNCAATourneyCompactResults.csv')
MNCAATourneyDetailedResults=pd.read_csv(os.getcwd()+r'\relevant_data_updated\MNCAATourneyDetailedResults.csv')

MNCAATourneySeeds=pd.read_csv(os.getcwd()+r'\relevant_data_updated\MNCAATourneySeeds.csv')

MRegularSeasonCompactResults=pd.read_csv(os.getcwd()+r'\relevant_data_updated\MRegularSeasonCompactResults.csv')
MRegularSeasonDetailedResults=pd.read_csv(os.getcwd()+r'\relevant_data_updated\MRegularSeasonDetailedResults.csv')

MTeams=pd.read_csv(os.getcwd()+r'\relevant_data_updated\MTeams.csv')

# Table by table

## MMasseyOrdinals

In [6]:
MMasseyOrdinals.shape

(4922512, 5)

In [7]:
MMasseyOrdinals.head()

,Season,RankingDayNum,SystemName,TeamID,OrdinalRank
0,2003,35,SEL,1102,159
1,2003,35,SEL,1103,229
2,2003,35,SEL,1104,12
3,2003,35,SEL,1105,314
4,2003,35,SEL,1106,260


In [8]:
MMasseyOrdinals['Season'].value_counts()

2019    328599
2017    315310
2020    299967
2023    298034
2018    292119
2014    292111
2016    291589
2022    284488
2013    282987
2015    281269
2012    248467
2011    232165
2010    223601
2021    219104
2009    183704
2008    169764
2005    139395
2007    139384
2004    136024
2006    132896
2003    131535
Name: Season, dtype: int64

In [9]:
MMasseyOrdinals['RankingDayNum'].value_counts()

133    332137
128    252723
100    244393
93     243944
107    234206
        ...  
54         25
75         25
89         25
103        25
53         25
Name: RankingDayNum, Length: 90, dtype: int64

In [10]:
set(MMasseyOrdinals[MMasseyOrdinals['Season']==2003]['SystemName']).intersection(set(MMasseyOrdinals[MMasseyOrdinals['Season']==2023]['SystemName']))

{'AP',
 'ARG',
 'BIH',
 'COL',
 'DOL',
 'DUN',
 'MAS',
 'MOR',
 'POM',
 'RTH',
 'SAG',
 'SEL',
 'USA',
 'WLK',
 'WOB',
 'WOL'}

In [11]:
MMasseyOrdinals['SystemName'].value_counts()[0:10]

SAG    130008
MOR    129018
POM    126527
DOK    113231
WLK    103919
MAS    102925
DOL    100721
WIL     98280
PGH     97016
COL     94106
Name: SystemName, dtype: int64

## MNCAATourneyCompactResults

In [12]:
MNCAATourneyCompactResults.shape

(2384, 8)

In [13]:
MNCAATourneyCompactResults.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0


## MNCAATourneyDetailedResults

In [14]:
MNCAATourneyDetailedResults.shape

(1248, 34)

In [15]:
MNCAATourneyDetailedResults.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,134,1421,92,1411,84,N,1,32,69,...,31,14,31,17,28,16,15,5,0,22
1,2003,136,1112,80,1436,51,N,0,31,66,...,16,7,7,8,26,12,17,10,3,15
2,2003,136,1113,84,1272,71,N,0,31,59,...,28,14,21,20,22,11,12,2,5,18
3,2003,136,1141,79,1166,73,N,0,29,53,...,17,12,17,14,17,20,21,6,6,21
4,2003,136,1143,76,1301,74,N,1,27,64,...,21,15,20,10,26,16,14,5,8,19


## MNCAATourneySeeds

In [16]:
MNCAATourneySeeds.shape

(2490, 3)

In [17]:
MNCAATourneySeeds.head()

,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374


In [18]:
MNCAATourneySeeds['Seed_number']=MNCAATourneySeeds['Seed'].str.extract('(\d+)')

In [19]:
MNCAATourneySeeds['Seed_number']=MNCAATourneySeeds['Seed_number'].apply(int)

In [20]:
MNCAATourneySeeds['Seed_number'].value_counts()

16    186
11    170
12    156
13    153
14    153
1     152
2     152
3     152
4     152
5     152
6     152
7     152
8     152
9     152
10    152
15    152
Name: Seed_number, dtype: int64

In [21]:
seeds=MNCAATourneySeeds[['Season', 'TeamID', 'Seed_number']]

## MRegularSeasonCompactResults

In [22]:
MRegularSeasonCompactResults.shape

(181682, 8)

In [23]:
MRegularSeasonCompactResults.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,20,1228,81,1328,64,N,0
1,1985,25,1106,77,1354,70,H,0
2,1985,25,1112,63,1223,56,H,0
3,1985,25,1165,70,1432,54,H,0
4,1985,25,1192,86,1447,74,H,0


## MRegularSeasonDetailedResults

In [24]:
MRegularSeasonDetailedResults.shape

(107634, 34)

In [25]:
MRegularSeasonDetailedResults.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14


## MTeams

In [26]:
MTeams.shape

(377, 4)

In [27]:
MTeams.head()

,TeamID,TeamName,FirstD1Season,LastD1Season
0,1101,Abilene Chr,2014,2023
1,1102,Air Force,1985,2023
2,1103,Akron,1985,2023
3,1104,Alabama,1985,2023
4,1105,Alabama A&M,2000,2023


In [28]:
MTeams.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377 entries, 0 to 376
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   TeamID         377 non-null    int64 
 1   TeamName       377 non-null    object
 2   FirstD1Season  377 non-null    int64 
 3   LastD1Season   377 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 11.9+ KB


# Data preparation

In [29]:
def winning_team(WTeamID, LTeamID, Season, DayNum):
    if WTeamID<LTeamID:
        return (1,WTeamID, LTeamID, Season, DayNum)
    else:
        return (0,LTeamID,WTeamID, Season, DayNum)        

In [30]:
target_df=pd.DataFrame(list(MNCAATourneyCompactResults.apply(lambda x: winning_team(x['WTeamID'], x['LTeamID'], x['Season'], x['DayNum']), axis=1)), columns=['Target','TeamID1', 'TeamID2', 'Season', 'DayNum'])

In [31]:
target_df

,Target,TeamID1,TeamID2,Season,DayNum
0,1,1116,1234,1985,136
1,1,1120,1345,1985,136
2,1,1207,1250,1985,136
3,1,1229,1425,1985,136
4,1,1242,1325,1985,136
...,...,...,...,...,...
2379,1,1242,1274,2022,146
2380,1,1314,1389,2022,146
2381,1,1242,1437,2022,152
2382,0,1181,1314,2022,152


## Same season rankings

In [32]:
systems={#'AP',
# 'ARG',
# 'BIH',
# 'COL',
# 'DOL',
# 'DUN',
 #'MAS',
 'MOR',
# 'POM',
 #'RTH',
 'SAG',
 #'SEL',
 #'USA',
 'WLK',
 #'WOB',
 #'WOL'
}

In [33]:
rankings=MMasseyOrdinals[MMasseyOrdinals['SystemName'].apply(lambda x: x in systems)].sort_values(['Season', 'TeamID','RankingDayNum'])

In [34]:
rankings=rankings.groupby(['TeamID', 'Season', 'SystemName'])['OrdinalRank'].agg([min, max, np.mean,'first', 'last']).reset_index().pivot(index=['TeamID','Season'], columns='SystemName').reset_index()

In [35]:
rankings.columns=rankings.columns.map('_'.join).str.strip('_')

In [36]:
rankings

,TeamID,Season,min_MOR,min_SAG,min_WLK,max_MOR,max_SAG,max_WLK,mean_MOR,mean_SAG,mean_WLK,first_MOR,first_SAG,first_WLK,last_MOR,last_SAG,last_WLK
0,1101,2014,348.0,329.0,310.0,350.0,350.0,336.0,349.210526,344.421053,326.750000,348.0,349.0,336.0,349.0,346.0,330.0
1,1101,2015,303.0,304.0,290.0,346.0,347.0,335.0,324.055556,324.555556,314.500000,303.0,314.0,335.0,346.0,336.0,332.0
2,1101,2016,311.0,320.0,303.0,336.0,347.0,327.0,327.666667,333.722222,311.000000,332.0,347.0,327.0,311.0,320.0,304.0
3,1101,2017,206.0,188.0,205.0,329.0,310.0,319.0,291.222222,277.611111,294.857143,309.0,310.0,205.0,317.0,305.0,307.0
4,1101,2018,213.0,244.0,226.0,274.0,313.0,269.0,246.000000,273.666667,244.733333,243.0,306.0,269.0,265.0,276.0,261.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7259,1473,2023,318.0,351.0,313.0,353.0,362.0,344.0,334.411765,355.000000,330.000000,353.0,362.0,316.0,342.0,355.0,338.0
7260,1474,2023,94.0,253.0,115.0,221.0,357.0,206.0,176.117647,284.000000,164.857143,102.0,357.0,159.0,200.0,253.0,191.0
7261,1475,2023,179.0,283.0,179.0,301.0,341.0,267.0,240.705882,315.352941,231.000000,194.0,341.0,201.0,280.0,314.0,267.0
7262,1476,2023,312.0,335.0,267.0,359.0,356.0,322.0,331.058824,342.352941,307.714286,359.0,356.0,267.0,328.0,336.0,318.0


## Same season compact results

### By team

In [37]:
MRegularSeasonCompactResults.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,20,1228,81,1328,64,N,0
1,1985,25,1106,77,1354,70,H,0
2,1985,25,1112,63,1223,56,H,0
3,1985,25,1165,70,1432,54,H,0
4,1985,25,1192,86,1447,74,H,0


In [38]:
def winning_team2(WTeamID, LTeamID, Season, DayNum, WScore, LScore, WLoc):
    
    def revert_home(x):
        if x=='H':
            return 'A'
        elif x=='A':
            return 'H'
        else:
            return x
        
    if WTeamID<LTeamID:
        return (1,WTeamID, LTeamID, Season, DayNum, WScore, LScore, WLoc)
    else:
        return (0,LTeamID,WTeamID, Season, DayNum, LScore, WScore, revert_home(WLoc))     


In [39]:
reg_season=pd.DataFrame(list(MRegularSeasonCompactResults.apply(lambda x: winning_team2(x['WTeamID'], x['LTeamID'], x['Season'], x['DayNum'], x['WScore'], x['LScore'], x['WLoc']), axis=1)), columns=['Win','TeamID1', 'TeamID2', 'Season', 'DayNum', 'Score1', 'Score2', 'WLoc'])

In [40]:
reg_season

,Win,TeamID1,TeamID2,Season,DayNum,Score1,Score2,WLoc
0,1,1228,1328,1985,20,81,64,N
1,1,1106,1354,1985,25,77,70,H
2,1,1112,1223,1985,25,63,56,H
3,1,1165,1432,1985,25,70,54,H
4,1,1192,1447,1985,25,86,74,H
...,...,...,...,...,...,...,...,...
181677,1,1104,1401,2023,132,82,63,N
181678,0,1222,1272,2023,132,65,75,H
181679,1,1343,1463,2023,132,74,65,H
181680,0,1336,1345,2023,132,65,67,N


In [41]:
reg_season_by_team_win=MRegularSeasonCompactResults.groupby(['Season', 'WTeamID']).agg({'NumOT':'count','WScore': 'sum'}).reset_index().rename(columns=
                                                                                                                  {'WTeamID':'TeamID','NumOT':'Wins', 'WScore':'Points'})

reg_season_by_team_loss=MRegularSeasonCompactResults.groupby(['Season', 'LTeamID']).agg({'NumOT':'count','LScore': 'sum'}).reset_index().rename(columns=
{'LTeamID':'TeamID','NumOT':'Losses', 'LScore':'Points'})


In [42]:
reg_season_by_team=reg_season_by_team_win.merge(reg_season_by_team_loss, how='left', on=['Season','TeamID'])

In [43]:
reg_season_by_team['Matches']=reg_season_by_team['Wins']+reg_season_by_team['Losses']
reg_season_by_team['Points']=reg_season_by_team['Points_x']+reg_season_by_team['Points_y']
reg_season_by_team['Avg_points_per_game']=reg_season_by_team['Points']/reg_season_by_team['Matches']

reg_season_by_team=reg_season_by_team[['Season','TeamID','Wins','Losses','Matches','Points','Avg_points_per_game']]

In [44]:
reg_season_by_team

,Season,TeamID,Wins,Losses,Matches,Points,Avg_points_per_game
0,1985,1102,5,19.0,24.0,1514.0,63.083333
1,1985,1103,9,14.0,23.0,1404.0,61.043478
2,1985,1104,21,9.0,30.0,2055.0,68.500000
3,1985,1106,10,14.0,24.0,1719.0,71.625000
4,1985,1108,19,6.0,25.0,2075.0,83.000000
...,...,...,...,...,...,...,...
12640,2023,1473,8,21.0,29.0,1961.0,67.620690
12641,2023,1474,15,15.0,30.0,2269.0,75.633333
12642,2023,1475,13,16.0,29.0,2199.0,75.827586
12643,2023,1476,13,17.0,30.0,1984.0,66.133333


### By pairs of teams

In [45]:
reg_season.head()

,Win,TeamID1,TeamID2,Season,DayNum,Score1,Score2,WLoc
0,1,1228,1328,1985,20,81,64,N
1,1,1106,1354,1985,25,77,70,H
2,1,1112,1223,1985,25,63,56,H
3,1,1165,1432,1985,25,70,54,H
4,1,1192,1447,1985,25,86,74,H


In [46]:
reg_season=reg_season.groupby(['TeamID1','TeamID2','Season'])['Win'].agg(['count',sum, np.mean]).reset_index().rename(columns={
    'count': 'together_matches_played',
    'sum':'together_number_of_wins',
    'mean': 'together_winning_percentage'
})

## Putting it all together

In [47]:
target_df.head()

,Target,TeamID1,TeamID2,Season,DayNum
0,1,1116,1234,1985,136
1,1,1120,1345,1985,136
2,1,1207,1250,1985,136
3,1,1229,1425,1985,136
4,1,1242,1325,1985,136


In [48]:
rankings.head()

,TeamID,Season,min_MOR,min_SAG,min_WLK,max_MOR,max_SAG,max_WLK,mean_MOR,mean_SAG,mean_WLK,first_MOR,first_SAG,first_WLK,last_MOR,last_SAG,last_WLK
0,1101,2014,348.0,329.0,310.0,350.0,350.0,336.0,349.210526,344.421053,326.750000,348.0,349.0,336.0,349.0,346.0,330.0
1,1101,2015,303.0,304.0,290.0,346.0,347.0,335.0,324.055556,324.555556,314.500000,303.0,314.0,335.0,346.0,336.0,332.0
2,1101,2016,311.0,320.0,303.0,336.0,347.0,327.0,327.666667,333.722222,311.000000,332.0,347.0,327.0,311.0,320.0,304.0
3,1101,2017,206.0,188.0,205.0,329.0,310.0,319.0,291.222222,277.611111,294.857143,309.0,310.0,205.0,317.0,305.0,307.0
4,1101,2018,213.0,244.0,226.0,274.0,313.0,269.0,246.000000,273.666667,244.733333,243.0,306.0,269.0,265.0,276.0,261.0


In [49]:
reg_season_by_team.head()

,Season,TeamID,Wins,Losses,Matches,Points,Avg_points_per_game
0,1985,1102,5,19.0,24.0,1514.0,63.083333
1,1985,1103,9,14.0,23.0,1404.0,61.043478
2,1985,1104,21,9.0,30.0,2055.0,68.500000
3,1985,1106,10,14.0,24.0,1719.0,71.625000
4,1985,1108,19,6.0,25.0,2075.0,83.000000


In [50]:
reg_season.head()

,TeamID1,TeamID2,Season,together_matches_played,together_number_of_wins,together_winning_percentage
0,1101,1102,2018,1,1,1.0
1,1101,1115,2015,1,1,1.0
2,1101,1116,2021,1,0,0.0
3,1101,1117,2018,1,0,0.0
4,1101,1117,2019,1,1,1.0


In [51]:
df=target_df.merge(rankings, how='left', left_on=['Season', 'TeamID1'], right_on=['Season', 'TeamID']).merge(rankings, how='left', left_on=['Season', 'TeamID2'], right_on=['Season', 'TeamID'])

In [52]:
df=df.drop(columns=['TeamID_x', 'TeamID_y'])

In [53]:
df=df.merge(reg_season_by_team, how='left', left_on=['Season', 'TeamID1'], right_on=['Season', 'TeamID']).merge(reg_season_by_team, how='left', left_on=['Season', 'TeamID2'], right_on=['Season', 'TeamID'])

In [54]:
df=df.drop(columns=['TeamID_x', 'TeamID_y'])

In [55]:
df=df.merge(reg_season, how='left', on=['Season', 'TeamID1','TeamID2'])

In [56]:
df=df.merge(seeds, how='left', left_on=['Season', 'TeamID1'], right_on=['Season', 'TeamID']).merge(seeds, how='left', left_on=['Season', 'TeamID2'], right_on=['Season', 'TeamID'])

In [57]:
df=df.drop(columns=['TeamID_x', 'TeamID_y'])

In [58]:
df.columns=[x.replace('_x', '_1').replace('_y','_2') for x in df.columns]

In [59]:
df

,Target,TeamID1,TeamID2,Season,DayNum,min_MOR_1,min_SAG_1,min_WLK_1,max_MOR_1,max_SAG_1,...,Wins_2,Losses_2,Matches_2,Points_2,Avg_points_per_game_2,together_matches_played,together_number_of_wins,together_winning_percentage,Seed_number_1,Seed_number_2
0,1,1116,1234,1985,136,NaN,NaN,NaN,NaN,NaN,...,20,10.0,30.0,2092.0,69.733333,1.0,0.0,0.0,9,8
1,1,1120,1345,1985,136,NaN,NaN,NaN,NaN,NaN,...,17,8.0,25.0,1728.0,69.120000,NaN,NaN,NaN,11,6
2,1,1207,1250,1985,136,NaN,NaN,NaN,NaN,NaN,...,11,18.0,29.0,1907.0,65.758621,NaN,NaN,NaN,1,16
3,1,1229,1425,1985,136,NaN,NaN,NaN,NaN,NaN,...,19,9.0,28.0,1915.0,68.392857,NaN,NaN,NaN,9,8
4,1,1242,1325,1985,136,NaN,NaN,NaN,NaN,NaN,...,20,7.0,27.0,1824.0,67.555556,NaN,NaN,NaN,3,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2379,1,1242,1274,2022,146,4.0,2.0,3.0,15.0,6.0,...,23,10.0,33.0,2469.0,74.818182,NaN,NaN,NaN,1,10
2380,1,1314,1389,2022,146,19.0,25.0,21.0,68.0,44.0,...,18,11.0,29.0,1919.0,66.172414,NaN,NaN,NaN,8,15
2381,1,1242,1437,2022,152,4.0,2.0,3.0,15.0,6.0,...,26,7.0,33.0,2397.0,72.636364,NaN,NaN,NaN,1,2
2382,0,1181,1314,2022,152,1.0,2.0,6.0,14.0,11.0,...,24,9.0,33.0,2558.0,77.515152,2.0,1.0,0.5,2,8


In [60]:
df['Avg_points_difference']=df['Avg_points_per_game_1']-df['Avg_points_per_game_2']
df['Avg_points_ratio']=df['Avg_points_per_game_1']/df['Avg_points_per_game_2']

df['Wins_difference']=df['Wins_1']/df['Wins_2']
df['Wins_percentage_ratio']=(df['Wins_1']*df['Matches_2'])/(df['Wins_2']*df['Matches_1'])

df['Seeds_difference']=df['Seed_number_1']-df['Seed_number_2']
df['Seeds_ratio']=df['Seed_number_1']/df['Seed_number_2']

In [61]:
df

,Target,TeamID1,TeamID2,Season,DayNum,min_MOR_1,min_SAG_1,min_WLK_1,max_MOR_1,max_SAG_1,...,together_number_of_wins,together_winning_percentage,Seed_number_1,Seed_number_2,Avg_points_difference,Avg_points_ratio,Wins_difference,Wins_percentage_ratio,Seeds_difference,Seeds_ratio
0,1,1116,1234,1985,136,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,9,8,-4.400000,0.936902,1.050000,0.954545,1,1.125000
1,1,1120,1345,1985,136,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,11,6,1.224828,1.017720,1.058824,0.912779,5,1.833333
2,1,1207,1250,1985,136,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1,16,9.982120,1.151799,2.272727,2.441077,-15,0.062500
3,1,1229,1425,1985,136,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,9,8,3.199735,1.046785,1.052632,1.091618,1,1.125000
4,1,1242,1325,1985,136,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3,14,8.477778,1.125493,1.150000,1.035000,-11,0.214286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2379,1,1242,1274,2022,146,4.0,2.0,3.0,15.0,6.0,...,NaN,NaN,1,10,3.770053,1.050390,1.217391,1.181586,-9,0.100000
2380,1,1314,1389,2022,146,19.0,25.0,21.0,68.0,44.0,...,NaN,NaN,8,15,11.342738,1.171412,1.333333,1.171717,-7,0.533333
2381,1,1242,1437,2022,152,4.0,2.0,3.0,15.0,6.0,...,NaN,NaN,1,2,5.951872,1.081941,1.076923,1.045249,-1,0.500000
2382,0,1181,1314,2022,152,1.0,2.0,6.0,14.0,11.0,...,1.0,0.5,2,8,2.661319,1.034333,1.166667,1.132353,-6,0.250000


In [62]:
df.to_csv('Dataset_for_modelling.csv', index=False)

## Dataset for scoring


In [63]:
result=[
]

for x in [2023]:#[2017,2018,2019,2021,2022]
    for y in MTeams['TeamID']:
        for z in MTeams['TeamID']:
            if y<z:
                result.append((x,y,z))

scoring=pd.DataFrame(result, columns=['Season', 'TeamID1', 'TeamID2'])

In [64]:
scoring=scoring.merge(rankings, how='left', left_on=['Season', 'TeamID1'], right_on=['Season', 'TeamID']).merge(rankings, how='left', left_on=['Season', 'TeamID2'], right_on=['Season', 'TeamID'])

In [65]:
scoring=scoring.drop(columns=['TeamID_x', 'TeamID_y'])

In [66]:
scoring=scoring.merge(reg_season_by_team, how='left', left_on=['Season', 'TeamID1'], right_on=['Season', 'TeamID']).merge(reg_season_by_team, how='left', left_on=['Season', 'TeamID2'], right_on=['Season', 'TeamID'])

In [67]:
scoring=scoring.drop(columns=['TeamID_x', 'TeamID_y'])

In [68]:
scoring=scoring.merge(reg_season, how='left', on=['Season', 'TeamID1','TeamID2'])

In [69]:
scoring=scoring.merge(seeds, how='left', left_on=['Season', 'TeamID1'], right_on=['Season', 'TeamID']).merge(seeds, how='left', left_on=['Season', 'TeamID2'], right_on=['Season', 'TeamID'])

In [70]:
scoring=scoring.drop(columns=['TeamID_x', 'TeamID_y'])

In [71]:
scoring.columns=[x.replace('_x', '_1').replace('_y','_2') for x in scoring.columns]

In [72]:
scoring

,Season,TeamID1,TeamID2,min_MOR_1,min_SAG_1,min_WLK_1,max_MOR_1,max_SAG_1,max_WLK_1,mean_MOR_1,...,Wins_2,Losses_2,Matches_2,Points_2,Avg_points_per_game_2,together_matches_played,together_number_of_wins,together_winning_percentage,Seed_number_1,Seed_number_2
0,2023,1101,1102,145.0,166.0,192.0,238.0,239.0,306.0,199.470588,...,14.0,18.0,32.0,2142.0,66.937500,NaN,NaN,NaN,NaN,NaN
1,2023,1101,1103,145.0,166.0,192.0,238.0,239.0,306.0,199.470588,...,20.0,11.0,31.0,2299.0,74.161290,NaN,NaN,NaN,NaN,NaN
2,2023,1101,1104,145.0,166.0,192.0,238.0,239.0,306.0,199.470588,...,29.0,5.0,34.0,2794.0,82.176471,NaN,NaN,NaN,NaN,1.0
3,2023,1101,1105,145.0,166.0,192.0,238.0,239.0,306.0,199.470588,...,12.0,18.0,30.0,2039.0,67.966667,NaN,NaN,NaN,NaN,NaN
4,2023,1101,1106,145.0,166.0,192.0,238.0,239.0,306.0,199.470588,...,7.0,23.0,30.0,1873.0,62.433333,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70871,2023,1474,1476,94.0,253.0,115.0,221.0,357.0,206.0,176.117647,...,13.0,17.0,30.0,1984.0,66.133333,NaN,NaN,NaN,NaN,NaN
70872,2023,1474,1477,94.0,253.0,115.0,221.0,357.0,206.0,176.117647,...,12.0,20.0,32.0,2261.0,70.656250,NaN,NaN,NaN,NaN,NaN
70873,2023,1475,1476,179.0,283.0,179.0,301.0,341.0,267.0,240.705882,...,13.0,17.0,30.0,1984.0,66.133333,NaN,NaN,NaN,NaN,NaN
70874,2023,1475,1477,179.0,283.0,179.0,301.0,341.0,267.0,240.705882,...,12.0,20.0,32.0,2261.0,70.656250,NaN,NaN,NaN,NaN,NaN


In [73]:
scoring['Avg_points_difference']=scoring['Avg_points_per_game_1']-scoring['Avg_points_per_game_2']
scoring['Avg_points_ratio']=scoring['Avg_points_per_game_1']/scoring['Avg_points_per_game_2']

scoring['Wins_difference']=scoring['Wins_1']/scoring['Wins_2']
scoring['Wins_percentage_ratio']=(scoring['Wins_1']*scoring['Matches_2'])/(scoring['Wins_2']*scoring['Matches_1'])

scoring['Seeds_difference']=scoring['Seed_number_1']-scoring['Seed_number_2']
scoring['Seeds_ratio']=scoring['Seed_number_1']/scoring['Seed_number_2']

In [74]:
scoring

,Season,TeamID1,TeamID2,min_MOR_1,min_SAG_1,min_WLK_1,max_MOR_1,max_SAG_1,max_WLK_1,mean_MOR_1,...,together_number_of_wins,together_winning_percentage,Seed_number_1,Seed_number_2,Avg_points_difference,Avg_points_ratio,Wins_difference,Wins_percentage_ratio,Seeds_difference,Seeds_ratio
0,2023,1101,1102,145.0,166.0,192.0,238.0,239.0,306.0,199.470588,...,NaN,NaN,NaN,NaN,4.254808,1.063564,0.642857,0.791209,NaN,NaN
1,2023,1101,1103,145.0,166.0,192.0,238.0,239.0,306.0,199.470588,...,NaN,NaN,NaN,NaN,-2.968983,0.959966,0.450000,0.536538,NaN,NaN
2,2023,1101,1104,145.0,166.0,192.0,238.0,239.0,306.0,199.470588,...,NaN,NaN,NaN,1.0,-10.984163,0.866334,0.310345,0.405836,NaN,NaN
3,2023,1101,1105,145.0,166.0,192.0,238.0,239.0,306.0,199.470588,...,NaN,NaN,NaN,NaN,3.225641,1.047459,0.750000,0.865385,NaN,NaN
4,2023,1101,1106,145.0,166.0,192.0,238.0,239.0,306.0,199.470588,...,NaN,NaN,NaN,NaN,8.758974,1.140293,1.285714,1.483516,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70871,2023,1474,1476,94.0,253.0,115.0,221.0,357.0,206.0,176.117647,...,NaN,NaN,NaN,NaN,9.500000,1.143649,1.153846,1.153846,NaN,NaN
70872,2023,1474,1477,94.0,253.0,115.0,221.0,357.0,206.0,176.117647,...,NaN,NaN,NaN,NaN,4.977083,1.070441,1.250000,1.333333,NaN,NaN
70873,2023,1475,1476,179.0,283.0,179.0,301.0,341.0,267.0,240.705882,...,NaN,NaN,NaN,NaN,9.694253,1.146586,1.000000,1.034483,NaN,NaN
70874,2023,1475,1477,179.0,283.0,179.0,301.0,341.0,267.0,240.705882,...,NaN,NaN,NaN,NaN,5.171336,1.073190,1.083333,1.195402,NaN,NaN


In [75]:
scoring.to_csv('Dataset_for_scoring.csv', index=False)